# Inicializar bibliotecas

In [1]:
!pip install -r requirements.txt


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Inicializar funções

In [7]:
import psycopg2
from faker import Faker
import random

# Função para criar tabelas
def create_tables():
    commands = (
        """
        CREATE TABLE tb_aluno (
            id_aluno SERIAL PRIMARY KEY,
            nome VARCHAR(255) NOT NULL,
            data_nascimento DATE NOT NULL,
            email VARCHAR(255) NOT NULL
        )
        """,
        """
        CREATE TABLE tb_curso (
            id_curso SERIAL PRIMARY KEY,
            nome VARCHAR(255) NOT NULL,
            descricao TEXT
        )
        """,
        """
        CREATE TABLE tb_disciplina (
            id_disciplina SERIAL PRIMARY KEY,
            nome VARCHAR(255) NOT NULL,
            creditos INT NOT NULL
        )
        """,
        """
        CREATE TABLE tb_turma (
            id_turma SERIAL PRIMARY KEY,
            id_disciplina INT NOT NULL,
            ano INT NOT NULL,
            semestre INT NOT NULL,
            FOREIGN KEY (id_disciplina)
                REFERENCES tb_disciplina (id_disciplina)
                ON UPDATE CASCADE ON DELETE CASCADE
        )
        """,
        """
        CREATE TABLE tb_matricula (
            id_aluno INT NOT NULL,
            id_curso INT NOT NULL,
            data_matricula DATE NOT NULL,
            PRIMARY KEY (id_aluno, id_curso),
            FOREIGN KEY (id_aluno)
                REFERENCES tb_aluno (id_aluno)
                ON UPDATE CASCADE ON DELETE CASCADE,
            FOREIGN KEY (id_curso)
                REFERENCES tb_curso (id_curso)
                ON UPDATE CASCADE ON DELETE CASCADE
        )
        """,
        """
        CREATE TABLE tb_inscricao (
            id_aluno INT NOT NULL,
            id_disciplina INT NOT NULL,
            data_inscricao DATE NOT NULL,
            PRIMARY KEY (id_aluno, id_disciplina),
            FOREIGN KEY (id_aluno)
                REFERENCES tb_aluno (id_aluno)
                ON UPDATE CASCADE ON DELETE CASCADE,
            FOREIGN KEY (id_disciplina)
                REFERENCES tb_disciplina (id_disciplina)
                ON UPDATE CASCADE ON DELETE CASCADE
        )
        """,
        """
        CREATE TABLE tb_turma_aluno (
            id_turma INT NOT NULL,
            id_aluno INT NOT NULL,
            status VARCHAR(50),
            PRIMARY KEY (id_turma, id_aluno),
            FOREIGN KEY (id_turma)
                REFERENCES tb_turma (id_turma)
                ON UPDATE CASCADE ON DELETE CASCADE,
            FOREIGN KEY (id_aluno)
                REFERENCES tb_aluno (id_aluno)
                ON UPDATE CASCADE ON DELETE CASCADE
        )
        """
    )
    try:
        conn = psycopg2.connect(host="localhost", dbname="gestao_universitaria_ETL", user="postgres", password="0710", port=5432)
        cur = conn.cursor()
        for command in commands:
            cur.execute(command)
        conn.commit()
        cur.close()
        print("Tabelas criadas com sucesso.")
    except (Exception, psycopg2.DatabaseError) as error:
        print(f"Erro ao criar tabelas: {error}")
    finally:
        if conn is not None:
            conn.close()


# Processo de popular tabelas
def populate_tables():
    try:
        conn = psycopg2.connect(host="localhost", dbname="postgres", user="postgres", password="jhis0909", port=5432)
        cur = conn.cursor()
        fake = Faker()
        
        #Definição dos cursos e descrições
        cursos = [
            ('Engenharia de Software', 'Curso que abrange o desenvolvimento de software, desde a análise de requisitos até a manutenção.'),
            ('Ciência da Computação', 'Curso focado nos fundamentos teóricos e práticos da computação e desenvolvimento de sistemas.'),
            ('Sistemas de Informação', 'Curso que combina TI com gestão, voltado para o desenvolvimento e gerenciamento de sistemas de informação.'),
            ('Análise e Desenvolvimento de Sistemas', 'Curso que prepara profissionais para projetar, desenvolver e implementar sistemas de software.'),
            ('Redes de Computadores', 'Curso voltado para a infraestrutura e comunicação entre computadores em redes.'),
            ('Inteligência Artificial', 'Curso focado em aprendizado de máquina, redes neurais, e outras áreas da IA.'),
            ('Engenharia de Controle e Automação', 'Curso voltado para automação industrial e sistemas de controle.'),
            ('Segurança da Informação', 'Curso que aborda a proteção de dados e sistemas contra ataques e acessos não autorizados.')
        ]

        # Inicialização das disciplinas e seus créditos
        disciplinas = [
            ('Fundamentos de Programação', 5),
            ('Estruturas de Dados e Algoritmos', 4),
            ('Banco de Dados', 4),
            ('Engenharia de Requisitos', 3),
            ('Arquitetura de Software', 4),
            ('Sistemas Operacionais', 4),
            ('Redes de Computadores', 3),
            ('Testes de Software', 3),
            ('Gestão de Projetos de Software', 3),
            ('Desenvolvimento Web', 5),
            ('Inteligência Artificial', 4),
            ('Segurança da Informação', 4),
            ('Engenharia de Software Ágil', 3),
            ('Desenvolvimento Mobile', 4),
            ('Machine Learning', 4)
        ]

        for nome_curso, descricao in cursos:
            cur.execute("""
                INSERT INTO tb_curso (nome, descricao)
                VALUES (%s, %s)
                RETURNING id_curso
                """, (nome_curso, descricao))
            
        for nome_disciplina, creditos in disciplinas:
            cur.execute("""
                INSERT INTO tb_disciplina (nome, creditos)
                VALUES (%s, %s)
                """, (nome_disciplina, creditos))

        #Populando database com 3 Milhões de alunos
        for _ in range(3000000):
            nome = fake.name()
            data_nascimento = fake.date_of_birth(minimum_age=18, maximum_age=40)
            email = fake.email()
            cur.execute("""
                INSERT INTO tb_aluno (nome, data_nascimento, email)
                VALUES (%s, %s, %s)
                RETURNING id_aluno
                """, (nome, data_nascimento, email))
            
            aluno_id = cur.fetchone()[0]
            
            curso_id = random.randint(1, len(cursos))
            data_matricula = fake.date_this_year()
            cur.execute("""
                INSERT INTO tb_matricula (id_aluno, id_curso, data_matricula)
                VALUES (%s, %s, %s)
                """, (aluno_id, curso_id, data_matricula))
            
            disciplina_id = random.randint(1, len(disciplinas))
            data_inscricao = fake.date_this_year()
            cur.execute("""
                INSERT INTO tb_inscricao (id_aluno, id_disciplina, data_inscricao)
                VALUES (%s, %s, %s)
                """, (aluno_id, disciplina_id, data_inscricao))

        # Populando database com 60 mil turmas
        turma_id = 1
        for disciplina_id in range(1, len(disciplinas)):
            for ano in [2023, 2024]:
                for _ in range(60000):
                    semestre = random.choice([1, 2])
                    cur.execute("""
                        INSERT INTO tb_turma (id_disciplina, ano, semestre)
                        VALUES (%s, %s, %s)
                        """, (disciplina_id, ano, semestre))
                    turma_id += 1

        for turma_id in range(1, 600001):
            alunos_turma = random.sample(range(1, 3000001), random.randint(50, 100))
            for aluno_id in alunos_turma:
                status = random.choice(['Ativo', 'Concluído'])
                cur.execute("""
                    INSERT INTO tb_turma_aluno (id_turma, id_aluno, status)
                    VALUES (%s, %s, %s)
                    """, (turma_id, aluno_id, status))

        conn.commit()
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

# Limpar todos os dados
def clean_tables():
    try:
        conn = psycopg2.connect(host="localhost", dbname="gestao_universitaria_ETL", user="postgres", password="0710", port=5432)
        cur = conn.cursor()
        truncate_sql = """TRUNCATE TABLE tb_turma_aluno, tb_inscricao, tb_matricula, tb_turma, tb_disciplina, tb_curso, tb_aluno RESTART IDENTITY CASCADE;"""
        cur.execute(truncate_sql)
        conn.commit()
        print("Tabelas limpas com sucesso")
        cur.close()
    
    except (Exception, psycopg2.DatabaseError) as error:
        print(f"Erro ao truncar tabelas: {error}")
    
    finally:
        if conn is not None:
            conn.close()
# Função para popular tabelas
def populate_tables():
    try:
        conn = psycopg2.connect(host="localhost", dbname="gestao_universitaria_ETL", user="postgres", password="0710", port=5432)
        cur = conn.cursor()
        fake = Faker()

        # Definição dos cursos e descrições
        cursos = [
            ('Engenharia de Software', 'Curso sobre desenvolvimento de software.'),
            ('Ciência da Computação', 'Curso sobre fundamentos de computação.'),
            ('Sistemas de Informação', 'Curso sobre TI e gestão.'),
            ('Análise e Desenvolvimento de Sistemas', 'Curso sobre desenvolvimento de sistemas.'),
            ('Redes de Computadores', 'Curso sobre redes e comunicação.'),
            ('Inteligência Artificial', 'Curso sobre IA e aprendizado de máquina.')
        ]

        # Inserir cursos
        for nome_curso, descricao in cursos:
            cur.execute("""
                INSERT INTO tb_curso (nome, descricao)
                VALUES (%s, %s)
                """, (nome_curso, descricao))

        # Definição das disciplinas e créditos
        disciplinas = [
            ('Fundamentos de Programação', 5),
            ('Estruturas de Dados e Algoritmos', 4),
            ('Banco de Dados', 4),
            ('Engenharia de Requisitos', 3)
        ]

        # Inserir disciplinas
        for nome_disciplina, creditos in disciplinas:
            cur.execute("""
                INSERT INTO tb_disciplina (nome, creditos)
                VALUES (%s, %s)
                """, (nome_disciplina, creditos))

        # População de alunos e matrícula
        for _ in range(100):  # Diminuído para 100 alunos para teste
            nome = fake.name()
            data_nascimento = fake.date_of_birth(minimum_age=18, maximum_age=40)
            email = fake.email()
            cur.execute("""
                INSERT INTO tb_aluno (nome, data_nascimento, email)
                VALUES (%s, %s, %s)
                RETURNING id_aluno
                """, (nome, data_nascimento, email))
            
            id_aluno = cur.fetchone()[0]
            id_curso = random.randint(1, len(cursos))
            data_matricula = fake.date_this_year()
            cur.execute("""
                INSERT INTO tb_matricula (id_aluno, id_curso, data_matricula)
                VALUES (%s, %s, %s)
                """, (id_aluno, id_curso, data_matricula))

            # Inscrição e turma
            id_disciplina = random.randint(1, len(disciplinas))
            data_inscricao = fake.date_this_year()
            cur.execute("""
                INSERT INTO tb_inscricao (id_aluno, id_disciplina, data_inscricao)
                VALUES (%s, %s, %s)
                """, (id_aluno, id_disciplina, data_inscricao))

        # Inserir turmas
        for id_disciplina in range(1, len(disciplinas) + 1):
            for ano in [2023, 2024]:
                for semestre in [1, 2]:
                    cur.execute("""
                        INSERT INTO tb_turma (id_disciplina, ano, semestre)
                        VALUES (%s, %s, %s)
                        """, (id_disciplina, ano, semestre))

        # Associar alunos a turmas
        for turma_id in range(1, 10):  # Para teste, associamos alunos em 10 turmas
            alunos_turma = random.sample(range(1, 101), random.randint(5, 10))
            for id_aluno in alunos_turma:
                status = random.choice(['Ativo', 'Concluído'])
                cur.execute("""
                    INSERT INTO tb_turma_aluno (id_turma, id_aluno, status)
                    VALUES (%s, %s, %s)
                    """, (turma_id, id_aluno, status))

        conn.commit()
        cur.close()
        print("Dados inseridos com sucesso.")
    except (Exception, psycopg2.DatabaseError) as error:
        print(f"Erro ao inserir dados: {error}")
    finally:
        if conn is not None:
            conn.close()




In [8]:
# Criar as tabelas
create_tables()


Tabelas criadas com sucesso.


In [9]:
#Popular as tabelas
populate_tables()

Dados inseridos com sucesso.


In [5]:
# Limpar as tabelas
clean_tables()

Tabelas limpas com sucesso
